In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import datetime
import joblib

# Load the dataset
dataset = pd.read_csv('new_dataset.csv')  # Replace with your dataset's path

# Extract features and targets
dataset['valid_time'] = pd.to_datetime(dataset['valid_time'])
dataset['day_of_year'] = dataset['valid_time'].dt.dayofyear
dataset['hour'] = dataset['valid_time'].dt.hour
processed_data = dataset.drop(columns=['valid_time', 'number', 'expver'])

X = processed_data[['day_of_year', 'hour']].values
y = processed_data.drop(columns=['day_of_year', 'hour']).values

# Normalize the input features (X) and target values (y)
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Save the scalers 
joblib.dump(scaler_X, "scaler_X.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

# Reshape X_scaled to a 3D array (samples, timesteps, features)
X_scaled = np.expand_dims(X_scaled, axis=1) 

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential([
    LSTM(128, activation='relu', input_shape=(X_scaled.shape[1], X_scaled.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(y_scaled.shape[1])  
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32,
    verbose=1
)

# Save the trained model
model_file_path = 'lstm_model.h5'
model.save(model_file_path)


# Define the prediction function
def predict_for_date(date_string, hour):
    # Convert date string to day_of_year
    input_date = datetime.datetime.strptime(date_string, "%Y-%m-%d")
    day_of_year = input_date.timetuple().tm_yday
    input_features = np.array([[day_of_year, hour]])  # Create a 2D array for input features

    # Normalize the input features using scaler_X
    input_features_scaled = scaler_X.transform(input_features)

    # Reshape to match LSTM input shape (samples, timesteps, features)
    input_features_scaled = np.expand_dims(input_features_scaled, axis=1)  # Add timestep dimension

    # Make predictions
    predictions_scaled = model.predict(input_features_scaled)

    # Rescale predictions back to the original range using scaler_y
    predictions_rescaled = scaler_y.inverse_transform(predictions_scaled)

    return predictions_rescaled[0]  # Return as a flat array


# Define the function to interpret climatic conditions
def interpret_climatic_conditions(predictions, columns):
    conditions = []
    predicted_values = dict(zip(columns, predictions))

    # Interpret temperature (convert from Kelvin to Celsius)
    temp = predicted_values.get("t2m", 0) - 273.15  # Convert Kelvin to Celsius
    if temp > 35:
        conditions.append("Heatwave warning")
    elif 0 <= temp <= 35:
        conditions.append("Normal temperature")
    elif temp < 0:
        conditions.append("Frost warning")

    # Interpret precipitation
    precip = predicted_values.get("tp", 0)  # Replace "tp" with actual precipitation column name
    if precip > 50:
        conditions.append("Heavy rainfall alert")
    elif 1 < precip <= 50:
        conditions.append("Clear weather with no chance of rainfall.")
    elif precip <= 1:
        conditions.append("Dry conditions")


Epoch 1/25
2300/2300 [==============================] - 13s 4ms/step - loss: 0.0530 - mae: 0.1548 - val_loss: 0.0427 - val_mae: 0.1349
Epoch 2/25
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0439 - mae: 0.1382 - val_loss: 0.0404 - val_mae: 0.1292
Epoch 3/25
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0414 - mae: 0.1320 - val_loss: 0.0382 - val_mae: 0.1230
Epoch 4/25
2300/2300 [==============================] - 11s 5ms/step - loss: 0.0389 - mae: 0.1248 - val_loss: 0.0374 - val_mae: 0.1200
Epoch 5/25
2300/2300 [==============================] - 11s 5ms/step - loss: 0.0382 - mae: 0.1224 - val_loss: 0.0375 - val_mae: 0.1182
Epoch 6/25
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0379 - mae: 0.1212 - val_loss: 0.0369 - val_mae: 0.1186
Epoch 7/25
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0377 - mae: 0.1206 - val_loss: 0.0369 - val_mae: 0.1186
Epoch 8/25
2300/2300 [==============================] -

C:\Users\Charu Gupta\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Function to interpret climatic conditions
def interpret_climatic_conditions(predictions, columns):
    conditions = []
    predicted_values = dict(zip(columns, predictions))

    # Interpret temperature (convert from Kelvin to Celsius)
    temp = predicted_values.get("t2m", 0) - 273.15  # Convert Kelvin to Celsius
    if temp > 35:
        conditions.append("Heatwave warning")
    elif 35 <= temp > 0:
        conditions.append("Normal temperature")
    elif temp < 0:
        conditions.append("Frost warning")

    # Interpret precipitation
    precip = predicted_values.get("tp", 0)  # Replace "tp" with actual precipitation column name
    if precip > 50:
        conditions.append("Heavy rainfall alert")
    elif 50 <= precip > 1:
        conditions.append("Clear weather no chance of rainfall.")
    elif precip < 1:
        conditions.append("Dry conditions")

    # Interpret wind
    wind_u = predicted_values.get("u10", 0)  # Replace "u10" with actual zonal wind column name
    wind_v = predicted_values.get("v10", 0)  # Replace "v10" with actual meridional wind column name
    wind_speed = (wind_u*2 + wind_v**2)*0.5  # Correct calculation for wind speed
    if wind_speed > 50:
        conditions.append("Strong wind advisory")
    else: 
        conditions.append("Normal wind speed")

    # Interpret pressure
    pressure = predicted_values.get("sp", 0)  # Replace "sp" with actual pressure column name
    if pressure < 1000:
        conditions.append("Low pressure: Possible storm")
    else:
        conditions.append("Normal pressure: No sign of storm")

    return conditions, temp  # Return conditions and temperature in Celsius

In [7]:
# user input
user_date = "2023-05-15"
user_hour = 12
user_prediction = predict_for_date(user_date, user_hour)

# Dynamically identify target column names (assuming already loaded dataset)
target_columns = list(processed_data.drop(columns=['day_of_year', 'hour']).columns)

# Interpret the climatic conditions
conditions, temperature_celsius = interpret_climatic_conditions(user_prediction, target_columns)

# Display the results
print(f"Predicted values for {user_date}, {user_hour}:00:")
print(f"Temperature (Celsius): {temperature_celsius:.2f}")
for condition in conditions:
    print(condition)



1/1 [==============================] - 0s 25ms/step
Predicted values for 2023-05-15, 12:00:
Temperature (Celsius): 24.51
Dry conditions
Normal wind speed
Normal pressure: No sign of storm
